In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS silver_crm;
CREATE SCHEMA IF NOT EXISTS silver_erp;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS silver_crm.silver_crm_cust_info (
  cst_id INT,
  cst_key STRING,
  cst_firstname STRING,
  cst_lastname STRING,
  cst_marital_status STRING,
  cst_gndr STRING,
  cst_create_date DATE
);

In [0]:
%sql
INSERT INTO silver_crm.silver_crm_cust_info (
  cst_id,
  cst_key,
  cst_firstname,
  cst_lastname,
  cst_marital_status,
  cst_gndr,
  cst_create_date
)
SELECT
  cst_id,
  cst_key,
  TRIM(cst_firstname) AS cst_firstname,
  TRIM(cst_lastname) AS cst_lastname,
  CASE 
    WHEN UPPER(TRIM(cst_marital_status)) = 'S' THEN 'Single'
    WHEN UPPER(TRIM(cst_marital_status)) = 'M' THEN 'Married'
    ELSE 'n/a'
  END AS cst_marital_status,
  CASE 
    WHEN UPPER(TRIM(cst_gndr)) = 'F' THEN 'Female'
    WHEN UPPER(TRIM(cst_gndr)) = 'M' THEN 'Male'
    ELSE 'n/a'
  END AS cst_gndr,
  cst_create_date
FROM (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY cst_id ORDER BY cst_create_date DESC) AS flag_last
  FROM bronze_crm.bronze_cust_info
  WHERE cst_id IS NOT NULL
) t
WHERE flag_last = 1;


In [0]:
%sql
SELECT COUNT(*) FROM silver_crm.silver_crm_cust_info;
SELECT * FROM silver_crm.silver_crm_cust_info LIMIT 10;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS silver_crm.silver_crm_prd_info (
  prd_id INT,
  cat_id STRING,
  prd_key STRING,
  prd_nm STRING,
  prd_cost INT,
  prd_line STRING,
  prd_start_dt DATE,
  prd_end_dt DATE
);

In [0]:
%sql
INSERT INTO silver_crm.silver_crm_prd_info (
  prd_id,
  cat_id,
  prd_key,
  prd_nm,
  prd_cost,
  prd_line,
  prd_start_dt,
  prd_end_dt
)
SELECT
  prd_id,
  REPLACE(SUBSTRING(prd_key, 1, 5), '-', '_') AS cat_id,
  SUBSTRING(prd_key, 7, LENGTH(prd_key)) AS prd_key,
  prd_nm,
  COALESCE(prd_cost, 0) AS prd_cost,
  CASE UPPER(TRIM(prd_line))
    WHEN 'M' THEN 'Mountain'
    WHEN 'R' THEN 'Road'
    WHEN 'S' THEN 'Other Sales'
    WHEN 'T' THEN 'Touring'
    ELSE 'n/a'
  END AS prd_line,
  CAST(prd_start_dt AS DATE) AS prd_start_dt,
  NULL AS prd_end_dt  -- Temporarily hardcoded
FROM workspace.bronze_crm.bronze_prd_info;

In [0]:
%sql
SELECT COUNT(*) FROM silver_crm.silver_crm_prd_info;
SELECT * FROM silver_crm.silver_crm_prd_info LIMIT 10;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS silver_crm.silver_crm_sales_details (
  sls_ord_num STRING,
  sls_prd_key STRING,
  sls_cust_id INT,
  sls_order_dt DATE,
  sls_ship_dt DATE,
  sls_due_dt DATE,
  sls_sales INT,
  sls_quantity INT,
  sls_price INT
);

In [0]:
%sql
INSERT INTO silver_crm.silver_crm_sales_details (
  sls_ord_num,
  sls_prd_key,
  sls_cust_id,
  sls_order_dt,
  sls_ship_dt,
  sls_due_dt,
  sls_sales,
  sls_quantity,
  sls_price
)
SELECT
  sls_ord_num,
  sls_prd_key,
  sls_cust_id,

  -- Order date cleansing
  CASE 
    WHEN sls_order_dt = 0 OR LENGTH(CAST(sls_order_dt AS STRING)) != 8 THEN NULL
    ELSE TO_DATE(CAST(sls_order_dt AS STRING), 'yyyyMMdd')
  END AS sls_order_dt,

  -- Ship date cleansing
  CASE 
    WHEN sls_ship_dt = 0 OR LENGTH(CAST(sls_ship_dt AS STRING)) != 8 THEN NULL
    ELSE TO_DATE(CAST(sls_ship_dt AS STRING), 'yyyyMMdd')
  END AS sls_ship_dt,

  -- Due date cleansing
  CASE 
    WHEN sls_due_dt = 0 OR LENGTH(CAST(sls_due_dt AS STRING)) != 8 THEN NULL
    ELSE TO_DATE(CAST(sls_due_dt AS STRING), 'yyyyMMdd')
  END AS sls_due_dt,

  -- Sales validation
  CASE 
    WHEN sls_sales IS NULL OR sls_sales <= 0 OR sls_sales != sls_quantity * ABS(sls_price)
      THEN sls_quantity * ABS(sls_price)
    ELSE sls_sales
  END AS sls_sales,

  sls_quantity,

  -- Price fix
  CASE 
    WHEN sls_price IS NULL OR sls_price <= 0 THEN 
      CASE WHEN sls_quantity = 0 THEN NULL ELSE sls_sales DIV sls_quantity END
    ELSE sls_price
  END AS sls_price

FROM workspace.bronze_crm.bronze_sales_details;

In [0]:
%sql
SELECT COUNT(*) FROM silver_crm.silver_crm_sales_details;
SELECT * FROM silver_crm.silver_crm_sales_details LIMIT 10;